In [2]:
import requests
import base64

# Replace with your Spotify credentials
CLIENT_ID = "a0723c247ed74e03b738798bc5494d47"
CLIENT_SECRET = "137f175bd88240808aad88925e38a88a"

# Function to get access token
def get_access_token():
    auth_url = "https://accounts.spotify.com/api/token"
    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()
    headers = {
        "Authorization": f"Basic {auth_header}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}

    response = requests.post(auth_url, headers=headers, data=data)
    response_data = response.json()
    
    return response_data.get("access_token")

# Function to get podcast (show) details
def get_podcast_info(show_id):
    access_token = get_access_token()
    if not access_token:
        print("Failed to retrieve access token.")
        return

    url = f"https://api.spotify.com/v1/shows/{show_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")

# Example: Replace with your podcast's show ID
show_id = "6t3lS8pB0XK9OyErfXS5aJ"  # Example show ID
podcast_info = get_podcast_info(show_id)

if podcast_info:
    print("Podcast Name:", podcast_info["name"])
    print("Publisher:", podcast_info["publisher"])
    print("Description:", podcast_info["description"])
    print("Total Episodes:", podcast_info["total_episodes"])


Podcast Name: Disordered: Anxiety Help
Publisher: Josh Fletcher and Drew Linsalata
Description: Disordered is the podcast that delivers real, evidence-based, actionable talk about anxiety disorders and anxiety recovery in a kind, compassionate, community-oriented environment. Josh Fletcher is a qualified psychotherapist in the UK. Drew Linsalata is a therapist practicing under supervision in the US. They're both bestselling authors in the anxiety and mental health space. Josh and Drew are funny, friendly, and they have a knack for combining lived experience, formal training, and professional experience in an encouraging, inspiring, and compassionate mental health message.
Total Episodes: 97


In [23]:
import pandas as pd
df=pd.read_excel('anxiety_resources.xlsx',sheet_name="podcasts")
df

,Podcast name,Description,Spotify,Apple
0,Disordered: Anxiety Help,NaN,https://open.spotify.com/show/6t3lS8pB0XK9OyEr...,https://podcasts.apple.com/us/podcast/disorder...
1,"The Anxious Truth - A Panic, Anxiety, and Ment...",NaN,https://open.spotify.com/show/3HWnUfK8VXBXVnrt...,https://podcasts.apple.com/us/podcast/the-anxi...
2,The Panic Pod,NaN,https://open.spotify.com/show/6SqjWK8zAD8nBe8p...,https://podcasts.apple.com/us/podcast/the-pani...


In [24]:
df['spotify_ids']=df['Spotify'].apply(lambda row : row.split('show/')[1])

In [25]:
def fetch_podcast_details(show_id):
    info = get_podcast_info(show_id)
    return {
        'description': info.get('description', ''),
        'publisher': info.get('publisher', '')
    } if info else {'description': '', 'publisher': ''}

# Apply function once and expand into multiple columns
df[['Description', 'Hosts']] = df['spotify_ids'].apply(fetch_podcast_details).apply(pd.Series)

,Podcast name,Description,Spotify,Apple,spotify_ids,Hosts
0,Disordered: Anxiety Help,"Disordered is the podcast that delivers real, ...",https://open.spotify.com/show/6t3lS8pB0XK9OyEr...,https://podcasts.apple.com/us/podcast/disorder...,6t3lS8pB0XK9OyErfXS5aJ,Josh Fletcher and Drew Linsalata
1,"The Anxious Truth - A Panic, Anxiety, and Ment...","Struggling with panic attacks, agoraphobia, or...",https://open.spotify.com/show/3HWnUfK8VXBXVnrt...,https://podcasts.apple.com/us/podcast/the-anxi...,3HWnUfK8VXBXVnrtJS7eAR,Drew Linsalata
2,The Panic Pod,Joshua is the author of best-selling books on ...,https://open.spotify.com/show/6SqjWK8zAD8nBe8p...,https://podcasts.apple.com/us/podcast/the-pani...,6SqjWK8zAD8nBe8pGOSFuu,Joshua Fletcher


In [19]:
df['Description']=df['spotify_ids'].apply(lambda row :get_podcast_info(row)['description'])
df['Hosts']=df['spotify_ids'].apply(lambda row :get_podcast_info(row)['publisher'])

In [22]:
df

,Podcast name,Description,Spotify,Apple,spotify_ids,description,Hosts
0,Disordered: Anxiety Help,"Disordered is the podcast that delivers real, ...",https://open.spotify.com/show/6t3lS8pB0XK9OyEr...,https://podcasts.apple.com/us/podcast/disorder...,6t3lS8pB0XK9OyErfXS5aJ,"Disordered is the podcast that delivers real, ...",Josh Fletcher and Drew Linsalata
1,"The Anxious Truth - A Panic, Anxiety, and Ment...","Struggling with panic attacks, agoraphobia, or...",https://open.spotify.com/show/3HWnUfK8VXBXVnrt...,https://podcasts.apple.com/us/podcast/the-anxi...,3HWnUfK8VXBXVnrtJS7eAR,"Struggling with panic attacks, agoraphobia, or...",Drew Linsalata
2,The Panic Pod,Joshua is the author of best-selling books on ...,https://open.spotify.com/show/6SqjWK8zAD8nBe8p...,https://podcasts.apple.com/us/podcast/the-pani...,6SqjWK8zAD8nBe8pGOSFuu,Joshua is the author of best-selling books on ...,Joshua Fletcher
